# Open your new rest

this repo includes insightful data analysis and visualizations on cafes and restaurants which are reviewed and rated on zomato in Bangkok



In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot


## PrePro

### read

In [2]:
df = pd.read_csv('./dbs/zomato.csv')

In [3]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [32]:
df.info()
# we can fill nulls in location column with adress column info and also fill rates with rates in review_list rates 
# dish liked nulls fill be filled with None value to imply nothing we can't drop null values in the column
# url adress phone columns are useless for our analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

#### Deal columns one by one

##### menu_item

In [10]:
df.loc[df['menu_item'] != '[]'].count()
# nearly 4/5 of columns don't have menu item info maybe it is better the drop menu_item column

url                            12100
address                        12100
name                           12100
online_order                   12100
book_table                     12100
rate                           11383
votes                          12100
phone                          11983
location                       12100
rest_type                      12060
dish_liked                      6852
cuisines                       12095
approx_cost(for two people)    12100
reviews_list                   12100
menu_item                      12100
listed_in(type)                12100
listed_in(city)                12100
dtype: int64

In [64]:
df['dish_liked'].iloc[1500]

float

##### reviews_list
we will check nan values in rate column and try to fill them with rates in reviews_list if rate is null and reviews_list = [] we will drop the row

In [3]:
df['rate'].isnull().value_counts()
# we have 7775 null rate

False    43942
True      7775
Name: rate, dtype: int64

In [4]:
df2= df.copy()

In [13]:
df2[df2['reviews_list']=='[]'].count()

url                            7595
address                        7595
name                           7595
online_order                   7595
book_table                     7595
rate                           2321
votes                          7595
phone                          7337
location                       7574
rest_type                      7525
dish_liked                      183
cuisines                       7565
approx_cost(for two people)    7543
reviews_list                   7595
menu_item                      7595
listed_in(type)                7595
listed_in(city)                7595
dtype: int64

In [5]:
prr = df[df['rate'].isna()]

In [6]:
len(prr)

7775

In [12]:
prr[prr['reviews_list'] != '[]'].count()
# we can fill 2501 of nan rate

url                            2501
address                        2501
name                           2501
online_order                   2501
book_table                     2501
rate                              0
votes                          2501
phone                          2348
location                       2501
rest_type                      2492
dish_liked                       27
cuisines                       2495
approx_cost(for two people)    2452
reviews_list                   2501
menu_item                      2501
listed_in(type)                2501
listed_in(city)                2501
dtype: int64

In [14]:
indi  = prr[prr['reviews_list'] == '[]' ].index
df2.drop(indi,inplace=True)

In [17]:
df2[df2['rate'].isna()].count()

url                            2501
address                        2501
name                           2501
online_order                   2501
book_table                     2501
rate                              0
votes                          2501
phone                          2348
location                       2501
rest_type                      2492
dish_liked                       27
cuisines                       2495
approx_cost(for two people)    2452
reviews_list                   2501
menu_item                      2501
listed_in(type)                2501
listed_in(city)                2501
dtype: int64

In [19]:
df2.reset_index(inplace=True)

In [18]:
df2['rate'].isna().value_counts()
# we are going to fill this 2501 rate row now

False    43942
True      2501
Name: rate, dtype: int64

In [49]:
df2[df2['rate'].isna()]

,index,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)


In [40]:
pat = re.compile('Rated [0-9]*\.[0-9]+',)
cs = []
for i in range(len(df2['reviews_list'])):

    rates = pat.findall(df2['reviews_list'][i])
    k = 0
    
    for rate in rates:
        k = k+int(rate.split('Rated ')[1].split('.')[0])
    if len(rates) != 0:
        k = k/len(rates)
        cs.append(k)
    else:
        cs.append(df2['reviews_list'][i])
    # [int(rate.split('Rated ')[1].split('.')[0]) for rate in rates]

In [ ]:
rateinde = df2['rate'].isna().index
for i in rateinde:
    df2['rate'][i] = cs[i]

In [50]:
df2['rate']

0         4.1/5
1         4.1/5
2         3.8/5
3         3.7/5
4         3.8/5
          ...  
46438       4.0
46439    2.5 /5
46440    3.6 /5
46441    4.3 /5
46442    3.4 /5
Name: rate, Length: 46443, dtype: object

In [51]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46443 entries, 0 to 46442
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   index                        46443 non-null  int64 
 1   url                          46443 non-null  object
 2   address                      46443 non-null  object
 3   name                         46443 non-null  object
 4   online_order                 46443 non-null  object
 5   book_table                   46443 non-null  object
 6   rate                         46443 non-null  object
 7   votes                        46443 non-null  int64 
 8   phone                        45458 non-null  object
 9   location                     46443 non-null  object
 10  rest_type                    46283 non-null  object
 11  dish_liked                   23636 non-null  object
 12  cuisines                     46426 non-null  object
 13  approx_cost(for two people)  46

In [150]:
1319982/46443

28.421548995542924

In [127]:
int(pat.findall(df2['reviews_list'][1])[1].split('Rated ')[1].split('.')[0])

3

In [121]:
re.search('Rated [0-9]*\.[0-9]+',df2['reviews_list'][1])

<re.Match object; span=(3, 12), match='Rated 4.0'>

In [ ]:
df2['reviews_list'][1]

In [ ]:
df['reviews_list'][1]
# reviews_list colum needs nlp